In [ ]:
#Prepatory Work Parts 2 & 3

#myRobot.py

from math import pi, sin, cos, sqrt, exp
from numpy import array, dot

from Robot3WD.Robot3WD import Robot

class myRobot(Robot):
    def __init__(self, sampling_period):
        Robot.__init__(self, sampling_period)

    def inverse_kinematics(self, p_dot, theta):
        L = self._L
        wheel_radius = self._wheel_radius
        M = array([[sin(theta), (-1)*cos(theta), (-1)*L], [cos((pi/6)+theta), sin((pi/6)+theta), (-1)*L], [(-1)*cos((pi/6)-theta), sin((pi/6)-theta), (-1)*L]])
        wheel_angular_velocities = (1/wheel_radius)*(dot(M, p_dot))
        return wheel_angular_velocities

    def forward_kinematics(self, wheel_angular_velocities, theta):
        L = self._L
        wheel_radius = self._wheel_radius
        M = array([[2*sin(theta), 2*cos(theta+(pi/6)), (-2)*sin(theta+(pi/3))], [-2*cos(theta), 2*cos(theta-(pi/3)), 2*cos(theta+(pi/3))], [(-1)/L, (-1)/L, (-1)/L]])
        p_dot = (wheel_radius/3)*(dot(M, wheel_angular_velocities))
        return p_dot

    def motor_limit(self, wheel_angular_velocities, limit_value):
    # Initialize the output list
      wheel_angular_velocities_bar = [0] * len(wheel_angular_velocities)

      for i in range(len(wheel_angular_velocities)):
          if wheel_angular_velocities[i] < -5 * limit_value:
              wheel_angular_velocities_bar[i] = -5 * limit_value
          elif -5 * limit_value <= wheel_angular_velocities[i] <= 5 * limit_value:
              wheel_angular_velocities_bar[i] = wheel_angular_velocities[i]
          elif wheel_angular_velocities[i] > 5 * limit_value:
              wheel_angular_velocities_bar[i] = 5 * limit_value

      return wheel_angular_velocities_bar


    def move_left(self, v, theta):
        p_dot = [-v, 0.0, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def move_forward(self, v, theta):
        p_dot = [0.0, v, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def move_backward(self, v, theta):
        p_dot = [0.0, -v, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def move_right(self, v, theta):
        p_dot = [v, 0.0, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def rotate_CCW(self, w, theta):
        p_dot = [0.0, 0.0, w] #<- positive since when testing on bot, with w set to negative the robot would rotate CW instead of CCW
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def rotate_CW(self, w, theta):
        p_dot = [0.0, 0.0, -w] #<- negative since when testing on bot, with w set to positive the robot would rotate CCW instead of CW
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

In [ ]:
from myRobot import myRobot
from Robot3WD.PID import PIDControllers
from numpy import array, dot, subtract, norm
from math import pi, sin, cos, sqrt, exp
# initialize sampling period & stopping time
T_s = 0.02
T_f = 20.0
robot = myRobot(T_s)

# initialize initial posture (x_0, y_0, theta_0)
x_0 = 0.0
y_0 = 0.0
theta_0 = pi/6

# initialize final posture (x_f, y_f, theta_f)
x_f = 0.7
y_f = 1.0
theta_f = pi/2

# initialize p_0, p_f, delta_p
p_0 = array([x_0, y_0, theta_0]).T
p_f = array([x_f, y_f, theta_f]).T
delta_p = subtract(p_f, p_0) #*

# initialize stopping criterion (epsilon)
epsilon = 0.01

# define the controller gains Kp, Ki and Kd
Kp = array([0.5, 0.5, 0.5]).T
Ki = array([0.2, 0.2, 0.2]).T
Kd = array([0.01, 0.01, 0.01]).T

# create an instance of the PID controllers object and initialize it
pid = PIDControllers(Kp, Ki, Kd, T_s)

# initialize robot & other variables (goal_reached, current_time, p, error_norm)
robot.initialize(theta_0)
goal_reached = False
current_time = 0.0
p = p_0
error_norm = norm(p_f - p)

# open a file to store data for plotting (using f.write (accuracy %10.3e))
f = open('p.csv') #*

# set start time to current time
start_time = time.time()

# Control loop
while ((not goal_reached) & (current_time < T_f)):

  # get current robot orientation (theta)
  robot.get_readings_update()
  theta = robot.orientation

  # determine p_dot (using forward kinematics)
  p_dot = (wheel_radius/3)*(dot(M, wheel_angular_velocities))

  # determine pd_dot (desired p_dot) using PID controller law
  # compute desired velocities of robot using the PID control law
  # p_f = final pose
  # p = actual pose of robot
  pd_dot = pid(p_f, p)

  # determine desired wheel velocities using inverse kinematics
  wheel_angular_velocities = (1/wheel_radius)*(dot(M, p_dot))

  # execute the motion
  # calls the function to send desired angular velocities for robot's wheels to the robot's motor controllers
  robot.set_angular_velocities(wheel_angular_velocities)

  # odometry update
  p = p + (p_dot*T_s)

  # replace calculated update with theta with measured value from IMU
  p[2] = robot.orientation

  # check to see if goal is reached
  if error_norm < epsilon:
    goal_reached = True

  # time update
  elapsed_time = time.time() - start_time

  # write data to file (current time, p[0], [1] ,p[2], error_norm (precision %10.3e))
  f.write('%10.3e %10.3e %10.3e %10.3e %10.3e\n' % (elapsed_time, p[0], p[1], p[2], error_norm) )
  # just some formatting rules, first part formats value of current time, uses tabs in between

# print the elapsed time
print("Elapsed time is", elapsed_time)

# Either goal is reached or current_time > T_f
if goal_reached:
  print("Goal is reached")
else:
  print("Goal is not reached")

robot.stop()
robot.close()

In [ ]:
from myRobot import myRobot
from Robot3WD.PID import PIDControllers
from numpy import array, dot, subtract, norm
from math import pi, sin, cos, sqrt, exp
import time

# initialize sampling period & stopping time
T_s = 0.02
T_f = 30.0
robot = myRobot(T_s)

# initialize initial posture (x_0, y_0, theta_0)
x_0 = 0.05
y_0 = -0.1
theta_0 = pi/6

# initialize final posture (x_f, y_f, theta_f)
x_f = 0.7
y_f = 1.0
theta_f = pi/2

# initialize p_0, p_f, delta_p
p_0 = array([x_0, y_0, theta_0]).T
p_f = array([x_f, y_f, theta_f]).T
delta_p = subtract(p_f, p_0) #*

# initialize stopping criterion (epsilon)
epsilon = 0.01

# define the controller gains Kp, Ki and Kd
Kp = array([5, 10, 5]).T
Ki = array([10, 20, 10]).T
Kd = array([0.01, 0.01, 0.01]).T

# create an instance of the PID controllers object and initialize it
pid = PIDControllers(Kp, Ki, Kd, T_s)

# initialize robot & other variables (goal_reached, current_time, p, error_norm)
robot.initialize(theta_0)
goal_reached = False
current_time = 0.0
p = p_0
error_norm = norm(p_f - p)

# open a file to store data for plotting (using f.write (accuracy %10.3e))
f = open('p.csv') #*

# set start time to current time
start_time = time.time()

# Control loop
while ((not goal_reached) & (current_time < T_f)):

  # get current robot orientation (theta)
  robot.get_readings_update()
  theta = robot.orientation

  # determine p_dot (using forward kinematics)
  p_dot = robot.forward_kinematics(robot.angular_velocities, theta)

  # determine pd_dot (desired p_dot) using PID controller law
  # compute desired velocities of robot using the PID control law
  # p_f = final pose
  # p = actual pose of robot
  pd_dot = pid(p_f, p)

  # determine desired wheel velocities using inverse kinematics
  wheel_velocities = robot.inverse_kinematics(pd_dot, theta)

  # execute the motion
  # calls the function to send desired angular velocities for robot's wheels to the robot's motor controllers
  robot.set_angular_velocities(wheel_angular_velocities)

  # odometry update
  p = p + (p_dot*T_s)

  # replace calculated update with theta with measured value from IMU
  p[2] = robot.orientation

  # check to see if goal is reached
  if error_norm < epsilon:
    goal_reached = True

  # time update
  elapsed_time = time.time() - start_time

  # write data to file (current time, p[0], [1] ,p[2], error_norm (precision %10.3e))
  f.write('%10.3e %10.3e %10.3e %10.3e %10.3e\n' % (elapsed_time, p[0], p[1], p[2], error_norm) )
  # just some formatting rules, first part formats value of current time, uses tabs in between

# print the elapsed time
print("Elapsed time is", current_time)

# Either goal is reached or current_time > T_f
if goal_reached:
  print("Goal is reached")
else:
  print("Goal is not reached")

robot.stop()
robot.close()

In [ ]:
#lab3main.py (WORKING)

from myRobot import myRobot
from Robot3WD.PID import PIDControllers
from numpy import array, dot, subtract
import numpy as np
from math import pi
import os
import time

# Initialize sampling period & stopping time
T_s = 0.02
T_f = 20.0
robot = myRobot(T_s)

# Initialize initial posture (x_0, y_0, theta_0)
x_0 = 0.0
y_0 = 0.0
theta_0 = pi / 6

# Initialize final posture (x_f, y_f, theta_f)
x_f = 0.7
y_f = 1.0
theta_f = pi / 2

# Initialize p_0, p_f
p_0 = array([x_0, y_0, theta_0]).T
p_f = array([x_f, y_f, theta_f]).T

# Initialize stopping criterion (epsilon)
epsilon = 0.01

# Define the controller gains Kp, Ki and Kd
Kp = array([0.5, 0.5, 0.5]).T
Ki = array([0.2, 0.2, 0.2]).T
Kd = array([0.01, 0.01, 0.01]).T

# Create an instance of the PID controllers object and initialize it
pid = PIDControllers(Kp, Ki, Kd, T_s)

# Initialize robot & other variables (goal_reached, current_time, p)
robot.initialize(theta_0)
goal_reached = False
current_time = 0.0
p = p_0

# Open a file to store data for plotting
with open('p.csv', 'w') as f:  # Use 'with' to ensure the file is properly closed
    # Set start time to current time
    start_time = time.time()

    # Control loop
    while not goal_reached and current_time < T_f:
        # Get current robot orientation (theta)
        robot.get_readings_update()
        theta = robot.orientation

        # Determine pd_dot (desired velocities) using PID controller
        pd_dot = pid(p_f, p)

        # Determine desired wheel velocities using inverse kinematics
        wheel_angular_velocities = robot.inverse_kinematics(pd_dot, theta)

        # Execute the motion by sending desired angular velocities to the robot's motors
        robot.set_angular_velocities(wheel_angular_velocities)

        # Update odometry
        p += pd_dot * T_s  # Update position based on desired velocities

        # Replace angle with the measured value from IMU
        p[2] = robot.orientation

        # Check to see if the goal is reached
        error_norm = np.linalg.norm(subtract(p_f, p))
        if error_norm < epsilon:
            goal_reached = True

        # Time update
        current_time = time.time() - start_time

        # Write data to file (current time, p[0], p[1], p[2], error_norm)
        f.write('%10.3e %10.3e %10.3e %10.3e %10.3e\n' % (current_time, p[0], p[1], p[2], error_norm))

# Print the elapsed time
print("Elapsed time is", current_time)

# Either goal is reached or current_time > T_f
if goal_reached:
    print("Goal is reached, error norm is:", error_norm)
else:
    print("Goal is not reached, error norm is:", error_norm)


In [ ]:
from myRobot import myRobot
from Robot3WD.PID import PIDControllers
from numpy import array, dot, subtract, sin, cos, arctan2
from math import pi
import time

# initialize sampling period & stopping time
T_s = 0.02
T_f = 30.0
f = 0.1  # Frequency for the desired trajectory
robot = myRobot(T_s)

# initialize initial posture (x_0, y_0, theta_0)
x_0 = 0.05
y_0 = -0.1
theta_0 = pi/6

# initialize p_0
p_0 = array([x_0, y_0, theta_0]).T

# define the controller gains Kp, Ki and Kd
Kp = array([5, 10, 5]).T
Ki = array([10, 20, 10]).T
Kd = array([0.01, 0.01, 0.01]).T

# create an instance of the PID controllers object and initialize it
pid = PIDControllers(Kp, Ki, Kd, T_s)

# initialize robot & other variables (goal_reached, current_time, p)
robot.initialize(theta_0)
current_time = 0.0
p = p_0

# open a file to store data for plotting (using f.write (accuracy %10.3e))
f = open('p.csv', 'w')  # Changed to write mode

# set start time to current time
start_time = time.time()

# Control loop
while current_time < T_f:
    # Compute desired trajectory at current time
    x_r = 0.5 * sin(f * current_time)
    y_r = 0.5 * sin(2 * f * current_time)
    x_r_dot = 0.5 * f * cos(f * current_time)
    y_r_dot = f * cos(2 * f * current_time)
    theta_r = arctan2(y_r_dot, x_r_dot)

    # Desired position and orientation
    p_r = array([x_r, y_r, theta_r])

    # get current robot orientation (theta)
    robot.get_readings_update()
    theta = robot.orientation

    # determine p_dot (using forward kinematics)
    p_dot = robot.forward_kinematics(robot.angular_velocities, theta)

    # determine pd_dot (desired p_dot) using PID controller law
    pd_dot = pid(p_r, p)

    # determine desired wheel velocities using inverse kinematics
    wheel_velocities = robot.inverse_kinematics(pd_dot, theta)

    # execute the motion
    robot.set_angular_velocities(wheel_velocities)

    # odometry update
    p = p + (p_dot * T_s)
    p[2] = robot.orientation  # update orientation from IMU

    # time update
    elapsed_time = time.time() - start_time
    current_time = elapsed_time

    # write data to file (current time, p[0], [1] ,p[2], error_norm (precision %10.3e))
    error_norm = norm(p_r - p)
    f.write('%10.3e %10.3e %10.3e %10.3e %10.3e\n' % (elapsed_time, p[0], p[1], p[2], error_norm))

# print the elapsed time
print("Elapsed time is", current_time)

# Close the robot and file after completion
robot.stop()
robot.close()
f.close()
